# Discrete Choice : Linear Models

## Definitions and Properties

**Random Utility Model (RUM):**

Let $u_{ij} = v_{ij} + \varepsilon_{ij}$, where $\varepsilon_{ij} \sim \text{i.i.d. Gumbel}(0, 1)$.

The agent $i$ chooses alternative $y_{i}$ such that $y_{i} = \argmax_{j} u_{ij}$.

In **conditional logit**, $v_{ij} = x_{j}^\top \beta$, where $x_{j}$ are characteristics of the alternatives (same for all individuals).

In **multinomial logit**, $v_{ij} = x_{i}^\top \beta_{j}$ where $x_{i}$ are characteristics of the individuals (same across alternatives).


**Conditional Choice Probability (CCP):**

$$
\Pr(d \mid v_{ij}) = \frac{\exp(v_{id})}{\sum_{j} \exp(v_{ij})}
$$



## Conditional Logit

## Multinomial Logit